In [7]:
%%writefile practice73.cu

// Practice 7 Task 3
// 1. Замерьте время выполнения редукции и сканирования для массивов разного размера.
// 2. Сравните производительность с CPU-реализацией.
// 3. Проведите оптимизацию кода, используя различные типы памяти CUDA.

#include <iostream>               // Для стандартного ввода-вывода
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL: ПАРАЛЛЕЛЬНАЯ РЕДУКЦИЯ
__global__ void reductionKernel(int *d_input, int *d_output, int n) {
    extern __shared__ int sdata[];            // Объявляем shared memory для блока потоков (быстрая память на блок)
    int tid = threadIdx.x;                    // Локальный индекс потока в блоке (0..blockDim.x-1)
    int i = blockIdx.x * blockDim.x + tid;    // Глобальный индекс потока в массиве данных

    // Загружаем данные в shared memory
    sdata[tid] = (i < n) ? d_input[i] : 0;    // Если индекс не выходит за пределы массива, копируем элемент, иначе ставим 0
    __syncthreads();                          // Синхронизация всех потоков блока перед началом редукции

    // Редукция в блоке: складываем элементы попарно
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) { // На каждом шаге уменьшаем количество активных потоков в 2 раза
        if (tid < s) sdata[tid] += sdata[tid + s];          // Складываем пару элементов
        __syncthreads();                                    // Синхронизация перед следующим шагом
    }

    // Записываем сумму блока в глобальную память
    if (tid == 0) d_output[blockIdx.x] = sdata[0];
}

// CUDA KERNEL: ПРЕФИКСНАЯ СУММА
__global__ void prefixSumKernel(int *d_input, int *d_output, int n) {
    extern __shared__ int temp[];                // Shared memory для блока потоков
    int tid = threadIdx.x;                       // Локальный индекс потока
    int i = blockIdx.x * blockDim.x + tid;       // Глобальный индекс элемента массива

    // Загружаем данные в shared memory
    temp[tid] = (i < n) ? d_input[i] : 0;        // Если вышли за предел массива, кладем 0
    __syncthreads();                             // Синхронизация потоков

    // Шаг вверх (upsweep) — строим дерево сумм
    for (int offset = 1; offset < blockDim.x; offset *= 2) {
        int index = (tid + 1) * offset * 2 - 1;  // Вычисляем индекс для текущей пары
        if (index < blockDim.x) temp[index] += temp[index - offset]; // Складываем элементы
        __syncthreads();                        // Синхронизация перед следующим шагом
    }

    // Шаг вниз (downsweep) — вычисляем префиксные суммы
    if (tid == 0) temp[blockDim.x - 1] = 0;      // Обнуляем последний элемент перед downsweep
    __syncthreads();                             // Синхронизация потоков

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        int index = (tid + 1) * offset * 2 - 1;  // Индекс текущей пары
        if (index < blockDim.x) {
            int t = temp[index - offset];        // Сохраняем значение для обмена
            temp[index - offset] = temp[index];  // Меняем местами
            temp[index] += t;                    // Складываем для префиксной суммы
        }
        __syncthreads();                        // Синхронизация перед следующим шагом
    }

    // Записываем результат обратно в глобальную память
    if (i < n) d_output[i] = temp[tid];
}

// ФУНКЦИЯ ДЛЯ ИЗМЕРЕНИЯ ВРЕМЕНи
void runPerformanceTest(int N) {
    cout << "\nРазмер массива: " << N << endl;   // Выводим размер массива

    int *h_array = new int[N];                   // Выделяем память для массива на CPU
    for (int i = 0; i < N; i++) h_array[i] = 1;  // Заполняем массив единицами

    // CPU: РЕДУКЦИЯ
    int cpu_sum = 0;                              // Переменная для суммы
    auto cpu_start = chrono::high_resolution_clock::now(); // Запуск таймера CPU
    for (int i = 0; i < N; i++) cpu_sum += h_array[i];     // Последовательная редукция
    auto cpu_end = chrono::high_resolution_clock::now();   // Остановка таймера CPU
    chrono::duration<double> cpu_time = cpu_end - cpu_start; // Вычисляем длительность

    // GPU: РЕДУКЦИЯ
    int *d_input, *d_intermediate;               // Указатели на GPU память
    cudaMalloc(&d_input, N * sizeof(int));       // Выделяем память на GPU для входного массива
    cudaMemcpy(d_input, h_array, N * sizeof(int), cudaMemcpyHostToDevice); // Копируем данные с CPU на GPU

    int threadsPerBlock = 256;                   // Количество потоков в блоке
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock; // Количество блоков в сетке
    cudaMalloc(&d_intermediate, blocksPerGrid * sizeof(int)); // Память под промежуточные суммы блоков

    cudaEvent_t gpu_start, gpu_stop;             // Таймер GPU
    cudaEventCreate(&gpu_start);                 // Создаём событие начала
    cudaEventCreate(&gpu_stop);                  // Создаём событие конца
    cudaEventRecord(gpu_start);                  // Старт таймера GPU

    reductionKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_intermediate, N);
                                                 // Запускаем ядро редукции

    int *h_intermediate = new int[blocksPerGrid];
    cudaMemcpy(h_intermediate, d_intermediate, blocksPerGrid * sizeof(int), cudaMemcpyDeviceToHost);
                                                 // Копируем промежуточные суммы блоков на CPU

    int gpu_sum = 0;
    for (int i = 0; i < blocksPerGrid; i++) gpu_sum += h_intermediate[i]; // Финальная редукция сумм блоков на CPU

    cudaEventRecord(gpu_stop);                   // Остановка таймера GPU
    cudaEventSynchronize(gpu_stop);              // Ждем завершения события
    float gpu_time = 0;
    cudaEventElapsedTime(&gpu_time, gpu_start, gpu_stop); // Время выполнения GPU редукции

    // CPU: СКАНИРОВАНИЕ
    int *h_prefix_cpu = new int[N];              // Массив для хранения префиксной суммы CPU
    auto cpu_scan_start = chrono::high_resolution_clock::now(); // Старт таймера
    h_prefix_cpu[0] = h_array[0];                // Первый элемент префиксной суммы равен первому элементу массива
    for (int i = 1; i < N; i++)                  // Последовательное вычисление префиксной суммы
        h_prefix_cpu[i] = h_prefix_cpu[i - 1] + h_array[i];
    auto cpu_scan_end = chrono::high_resolution_clock::now();   // Остановка таймера
    chrono::duration<double> cpu_scan_time = cpu_scan_end - cpu_scan_start; // Время CPU сканирования

    // GPU: СКАНИРОВАНИЕ
    int *d_output;                               // Память на GPU для результата сканирования
    cudaMalloc(&d_output, N * sizeof(int));
    cudaEventRecord(gpu_start);                  // Старт таймера
    prefixSumKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, N);
                                                 // Запуск ядра сканирования
    cudaEventRecord(gpu_stop);                   // Стоп таймера
    cudaEventSynchronize(gpu_stop);              // Синхронизация
    float gpu_scan_time = 0;
    cudaEventElapsedTime(&gpu_scan_time, gpu_start, gpu_stop); // Время выполнения GPU сканирования

    // Вывод результатов
    cout << "CPU время (редукция): " << cpu_time.count() * 1000 << " мс" << endl;          // Время CPU редукции
    cout << "GPU время (редукция): " << gpu_time << " мс" << endl;                         // Время GPU редукции
    cout << "CPU время (сканирование): " << cpu_scan_time.count() * 1000 << " мс" << endl; // Время CPU сканирования
    cout << "GPU время (сканирование): " << gpu_scan_time << " мс" << endl;                // Время GPU сканирования

    // Освобожденние памяти
    delete[] h_array;                             // Освобождаем массив CPU
    delete[] h_intermediate;                      // Освобождаем промежуточный массив CPU
    delete[] h_prefix_cpu;                        // Освобождаем массив CPU для сканирования
    cudaFree(d_input);                            // Освобождаем память GPU
    cudaFree(d_intermediate);
    cudaFree(d_output);
}

// Основная функция
int main() {
    runPerformanceTest(1024);       // Тестируем массив из 1024 элементов
    runPerformanceTest(1000000);    // Тестируем массив из 1 000 000 элементов
    runPerformanceTest(10000000);   // Тестируем массив из 10 000 000 элементов
    return 0;                        // Завершаем программу
}



Overwriting practice73.cu


In [8]:
# Компиляция
!nvcc practice73.cu -o practice73 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                    # -std=c++11 — стандарт C++
# Запуск
!./practice73


Размер массива: 1024
CPU время (редукция): 0.003553 мс
GPU время (редукция): 0.153696 мс
CPU время (сканирование): 0.004041 мс
GPU время (сканирование): 0.028576 мс

Размер массива: 1000000
CPU время (редукция): 2.88083 мс
GPU время (редукция): 0.140608 мс
CPU время (сканирование): 5.14558 мс
GPU время (сканирование): 0.321344 мс

Размер массива: 10000000
CPU время (редукция): 27.0933 мс
GPU время (редукция): 1.26176 мс
CPU время (сканирование): 47.2609 мс
GPU время (сканирование): 3.04918 мс
